In [2]:
import pandas as pd
import numpy as np

In [3]:
df = pd.read_csv("../data/Super_Store_data.csv",parse_dates=["Order Date","Ship Date"])

In [4]:
df['Region']=df['Region'].astype('category')

In [5]:
df['Sub-Category']=df['Sub-Category'].astype('category')

In [6]:
df = df[df["Coordinates"].str.contains(",")]

In [7]:
df["Coordinates"] =df["Coordinates"].str.replace("(","",regex=False).str.replace(")","",regex=False)

In [8]:
df[['lat', 'long']]=df["Coordinates"].str.split(",",expand=True)

In [9]:
df.drop(columns=["Coordinates"],inplace=True)

In [10]:
df["price"]= df["Sales"]/df["Quantity"]

In [11]:
prom = df.loc[df["Sales"]!=1].groupby(["Product ID"])["price"].agg(["mean"]).reset_index()

In [12]:
sales_w=df.loc[df["Sales"]==1]
df=df.loc[df["Sales"]!=1]

In [13]:
sales_w=sales_w.merge(prom, how="left",on="Product ID")

In [14]:
sales_w["Sales"]=sales_w["Quantity"]*sales_w["mean"]

In [15]:
sales_w=sales_w.loc[:,df.columns]

In [16]:
df = pd.concat([df,sales_w])

In [17]:
df.shape

(2063, 24)

In [18]:
# retirar profits negativos
df = df.loc[df["Profit"]>0]

In [19]:
df = df.loc[~(df["Profit"]>df["Sales"])]

In [20]:
#valores atipicos
def valor_atipico(df,x):
    q3=df[x].quantile(0.75)
    q1=df[x].quantile(0.25)
    iqr=q3-q1
    print ("Cantidad de valores atipicos de la columna "+x+": ",df.loc[(df[x]>q3+1.5*iqr)|(df[x]<q1-1.5*iqr)].shape[0])
    df_nuevo = df.loc[(df[x]<=q3+1.5*iqr)&(df[x]>=q1-1.5*iqr)]
    return df_nuevo

In [21]:
# valores atipicos
df_nuevo= valor_atipico(df,"Sales")

Cantidad de valores atipicos de la columna Sales:  166


In [22]:
df_nuevo.shape

(1148, 24)

In [23]:
df_nuevo["lat"] = df_nuevo["lat"].astype(float)
df_nuevo["long"] = df_nuevo["long"].astype(float)
df_nuevo.drop(columns=["price"],inplace=True)

/tmp/ipykernel_121184/2759517915.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_nuevo["lat"] = df_nuevo["lat"].astype(float)
/tmp/ipykernel_121184/2759517915.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_nuevo["long"] = df_nuevo["long"].astype(float)
/tmp/ipykernel_121184/2759517915.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#retur

In [24]:
df_nuevo.dtypes

Row ID                    int64
Order ID                 object
Order Date       datetime64[ns]
Ship Date        datetime64[ns]
Ship Mode                object
Customer ID              object
Customer Name            object
Segment                  object
Country                  object
City                     object
State                    object
Postal Code               int64
Region                 category
Product ID               object
Category                 object
Sub-Category           category
Product Name             object
Sales                   float64
Quantity                  int64
Discount                float64
Profit                  float64
lat                     float64
long                    float64
dtype: object

In [25]:
df_nuevo["diff date"] = df_nuevo["Ship Date"]-df_nuevo["Order Date"]

/tmp/ipykernel_121184/1244729463.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_nuevo["diff date"] = df_nuevo["Ship Date"]-df_nuevo["Order Date"]


In [26]:
dias =df_nuevo.loc[df_nuevo["Order Date"]>df_nuevo["Ship Date"]]

In [27]:
df_nuevo=df_nuevo.loc[~(df_nuevo["Order Date"]>df_nuevo["Ship Date"])]

In [28]:
ship_mode = df_nuevo.groupby("Ship Mode")["diff date"].mean().reset_index()

In [29]:
ship_mode

,Ship Mode,diff date
0,First Class,2 days 00:49:39.310344827
1,Same Day,0 days 00:30:38.297872340
2,Second Class,2 days 22:50:26.086956521
3,Standard Class,4 days 13:40:57.318952234


In [30]:
dias = dias.merge(ship_mode,how="left",on="Ship Mode")
dias["Ship Date"]=dias["Order Date"]+dias["diff date_y"]

In [31]:
dias["Ship Date"]= dias["Ship Date"].dt.date

In [32]:
df_nuevo = pd.concat([df_nuevo.iloc[:,:-1],dias.iloc[:,:-2]])

In [33]:
df_nuevo.dtypes

Row ID                    int64
Order ID                 object
Order Date       datetime64[ns]
Ship Date                object
Ship Mode                object
Customer ID              object
Customer Name            object
Segment                  object
Country                  object
City                     object
State                    object
Postal Code               int64
Region                 category
Product ID               object
Category                 object
Sub-Category           category
Product Name             object
Sales                   float64
Quantity                  int64
Discount                float64
Profit                  float64
lat                     float64
long                    float64
dtype: object

In [35]:
df_nuevo.to_csv("../data/clean_data.csv")